# GameDay2020 Notebook

The standard dilemma of fantasy baseball is who to choose next: the best available player but in a position you've already filled?; the best at a rare position?; the next best closer because they're going fast?  Whatever choice you make will have consequences, as the next player will no longer have your pick as a choice, and so on.  Ideally you would forecast how each of those would play out and take the best option --- that's exactly what this code does: it identifies the best player left in each of the positions you still need to fill, simulates the rest of draft in each of those scenarios, and returns the best choice.

And it's easy to use!  After [cloning](https://github.com/wrapgenius/GameDay2020) GameDay2020 (and provided you have Jupyter Notebooks with Python 3 installed, and a spreadsheet program) you should have everything you need to do a live roto draft.

The code is based on three ingredients: *projections*, *rankings*, and *injuries*:  
- Projections are estimates of player performances for 2020, and are based on data scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) (model = ZiPS, Steamer, or TheBat.) 
- Rankings are draft-order recommendations.  The default (i.e.; ranking_method = 'Yahoo') is to use Yahoo Rankings (from Feb 22, 2020).  Others include [ESPN](https://www.espn.com/fantasy/baseball/story/_/id/28285423/fantasy-baseball-2020-rankings-roto-rotisserie), [FantasyPros](https://www.fantasypros.com/mlb/rankings/overall.php), and [Rotographs](https://fantasy.fangraphs.com), or you can put in your own.  Yahoo for yahoo leagues makes most sense since it most closely imitates autodraft.    
- Injured players to be removed from draft consideration are located in the Injuries2020.xlsx file found in the Injured_List_Spreadsheets directory. 

_Your_ roto league may have more or less than 12 teams, or use different stats; that ok!  Declare them when defining the Draft object: most of the obvious stats are included, more obscure ones may require hacking into the [fangraphs_projection_2020](https://github.com/wrapgenius/GameDay2020/blob/master/GameDayFunctions/fangraphs_projection_2020.py).  

Note, if you choose to shuffle_picks = True, it will not default to the next best pick, but draw a random number between 0 and 1 and compare it to the sigmoid function 1/1+exp(-(pick_number - avg_pick_number)).  So say it's the 15th pick and Trevor Story, who is typically drafted 10th is available, there is an 85% chance you'll select him.  Also note, it can't _yet_ do AL or NL only. 

Below are examples of the different ways the code can be run.  

In [1]:
# Load packages.  Must have pandas and numpy.  

import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from GameDayFunctions.fangraphs_projection_2020 import Projection
from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"
shuffle_picks = True

First need a **Projection** object.  Projections are scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips), and include ZiPS, Steamer, and TheBat.  
In this example I create a dictionary of the three projections.  

In [24]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = 'FantasyPros')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer',ranking_method = 'Yahoo')
player_ranking_thebat = Projection(path_data=path_data,year=year,model='TheBat',ranking_method = 'ESPN')
player_rankings = {"ZiPS":player_ranking_zips, "Steamer":player_ranking_steam, "TheBat":player_ranking_thebat}

Next need a **Draft** object.  The default, e.g.; Draft(player_rankings['ZiPS']), uses 12 teams and the roster spots/roto stats shown in this example. 

In [4]:
# Enter your roto-league-specific stats.  The default settings are shown here.  
draft_position = 2
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 

# Choose the projection system you prefer ()
player_projections = player_rankings['ZiPS']

# Get an instance of the Draft Class with your league-specific details and projection preference.
zips_example = Draft(player_projections, 
                     draft_position = draft_position, 
                     number_teams = number_teams,
                     roster_spots = roster_spots,
                     batter_stats = batter_stats,
                     pitcher_stats = pitcher_stats)

To use while drafting in real-time, simply record the draft results in a spreadsheet (.xlsx, so if you're using Numbers, need to export to Excel) and run the cell. 

Examples below show results for after 2, 3, and 4 rounds.  Notice that at each stage the projected placement in the standings improves.  

In [5]:
# Example pick in the 2nd round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround2.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Juan Soto for OF
Team 9 picking Cody Bellinger for OF
Team 10 picking Ronald Acuna Jr. for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Fernando Tatis Jr. for SS
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Finding Best Pick For Team 5
Stored Result for Pick 0 Gerrit Cole SP whose placing/score is [11, 100.5]
Stored Result for Pick 1 Trea Turner SS whose placing/score is [12, 123.5]
Stored Result for Pick 2 Walker Buehler P whose placing/score is [11, 98.5]
Stored Result for Pick 3 J.D. Martinez OF w

In [6]:
# Example pick in the 3rd round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround3.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Juan Soto for OF
Team 9 picking Cody Bellinger for OF
Team 10 picking Ronald Acuna Jr. for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Fernando Tatis Jr. for SS
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Team 5 picking Trea Turner for SS
Team 4 picking Bryce Harper for OF
Team 3 picking Chris Sale for SP
Team 2 picking Walker Buehler for SP
Team 1 picking Shane Bieber for SP
Team 1 picking Stephen Strasburg for SP
Team 2 picking Josh Hader for RP
Team 3 picking Rafael Devers for 3B
Team 4 picking 

In [7]:
# Example pick in the 4th round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround4.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Gerrit Cole for SP
Team 9 picking Cody Bellinger for OF
Team 10 picking Juan Soto for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Ronald Acuna Jr. for OF
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Team 5 picking Trea Turner for SS
Team 4 picking Bryce Harper for OF
Team 3 picking Chris Sale for SP
Team 2 picking Walker Buehler for SP
Team 1 picking Shane Bieber for SP
Team 1 picking Stephen Strasburg for SP
Team 2 picking Josh Hader for RP
Team 3 picking Rafael Devers for 3B
Team 4 picking Fernand

In [8]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = projected_roto_stats[0]
projected_standings = projected_roto_stats[3]
projected_place = projected_roto_stats[4]
projected_scores = projected_roto_stats[5]

In [9]:
# The projected_stats give the raw numbers

projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,664.0,747.0,231.0,26.0,181.0,640.0,111.0,442.0,0.270591,0.801498,83.0,50.0,6.0,6.0,0.0,310.0,1296.0,3.595280,1.155854,0.0
1,698.0,708.0,252.0,23.0,245.0,789.0,87.0,432.0,0.269399,0.833192,69.0,47.0,5.0,5.0,38.0,315.0,1050.0,3.974523,1.229275,3.0
2,886.0,863.0,297.0,33.0,268.0,871.0,109.0,593.0,0.272972,0.839089,57.0,34.0,4.0,4.0,69.0,216.0,868.0,3.536265,1.158818,6.0
3,703.0,714.0,234.0,34.0,231.0,777.0,105.0,481.0,0.268373,0.827850,50.0,35.0,2.0,2.0,25.0,269.0,848.0,3.958393,1.261790,2.0
4,801.0,867.0,283.0,50.0,230.0,785.0,170.0,498.0,0.270470,0.809724,65.0,43.0,5.0,5.0,25.0,311.0,1049.0,3.578203,1.181481,2.0
5,743.0,776.0,277.0,26.0,248.0,820.0,77.0,486.0,0.267882,0.823483,75.0,47.0,6.0,6.0,24.0,292.0,1203.0,3.445659,1.130363,2.0
6,649.0,742.0,258.0,30.0,183.0,656.0,90.0,454.0,0.276528,0.821493,43.0,31.0,2.0,2.0,56.0,200.0,730.0,3.837765,1.214583,4.0
7,807.0,775.0,252.0,27.0,269.0,855.0,63.0,613.0,0.268190,0.848362,50.0,33.0,2.0,2.0,75.0,187.0,777.0,3.565066,1.138329,6.0
8,829.0,827.0,285.0,30.0,288.0,899.0,79.0,559.0,0.275407,0.857130,61.0,41.0,5.0,5.0,29.0,269.0,1028.0,3.634032,1.179834,2.0
9,810.0,872.0,271.0,25.0,231.0,782.0,127.0,528.0,0.281279,0.837224,63.0,49.0,3.0,3.0,31.0,333.0,1062.0,3.838502,1.235548,3.0


In [10]:
# And projected_scores show how they translate to roto scores

projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,2.0,5.0,1.0,4.0,1.0,1.0,9.0,2.0,7.0,1.0,12.0,1.0,11.5,11.5,1.0,5.0,12.0,7.0,9.0,12.0
1,3.0,2.0,4.5,1.0,7.5,8.0,5.0,1.0,5.0,8.0,9.0,3.5,9.0,9.0,8.0,2.0,7.0,1.0,3.0,5.5
2,12.0,10.0,12.0,10.0,10.0,11.0,8.0,11.0,8.0,10.0,4.0,10.0,6.0,6.0,10.0,10.0,4.0,10.0,8.0,2.0
3,4.0,3.0,2.0,11.0,5.5,5.0,7.0,4.0,4.0,6.0,2.5,9.0,2.0,2.0,4.5,8.5,3.0,2.0,1.0,8.5
4,7.0,11.0,9.0,12.0,3.5,7.0,12.0,6.0,6.0,2.0,8.0,6.0,9.0,9.0,4.5,3.5,6.0,8.0,6.0,8.5
5,6.0,7.0,8.0,4.0,9.0,9.0,3.0,5.0,2.0,4.0,11.0,3.5,11.5,11.5,3.0,6.0,11.0,12.0,11.0,8.5
6,1.0,4.0,6.0,8.0,2.0,2.0,6.0,3.0,11.0,3.0,1.0,12.0,2.0,2.0,9.0,11.0,1.0,4.0,4.0,4.0
7,8.0,6.0,4.5,6.0,11.0,10.0,2.0,12.0,3.0,11.0,2.5,11.0,2.0,2.0,11.5,12.0,2.0,9.0,10.0,2.0
8,11.0,8.0,10.5,8.0,12.0,12.0,4.0,10.0,10.0,12.0,5.0,8.0,9.0,9.0,6.0,8.5,5.0,6.0,7.0,8.5
9,9.0,12.0,7.0,2.0,5.5,6.0,10.0,8.0,12.0,9.0,7.0,2.0,4.0,4.0,7.0,1.0,8.0,3.0,2.0,5.5


In [11]:
# Finally, the projected_standings shows the resulting standings

projected_standings

2     172.0
8     169.5
5     146.0
4     144.0
11    141.5
7     137.5
9     124.0
10    118.0
0     115.0
1     102.0
6      96.0
3      94.5
dtype: float64

In [12]:
# But if you just want to know how your team finishes, that's projected_place

projected_place

4

### You can simulate drafts as well. Naive drafts simply picks the next eligible (i.e., open roster spot) player on the ranked list. 

In [25]:
# Try a quick draft: 4 teams, limited roster spots, naive (i.e., next available pick on ranked list) picks.

zipsS= Draft(player_rankings['ZiPS'],draft_position = 2,number_teams = 4, roster_spots = {'C':0,'1B':0,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':1,'RP':1,'P':1,'BN':1})
zipsS.draft_all(naive_draft = True, shuffle_picks = shuffle_picks)
print('Placement = '+str(zipsS.roto_placement))

Placement = 4


In [26]:
# Now try a naive draft again, but for full 12 team and all 24 roster spots.  Draft Position is 2 

zipsNaive_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zipsNaive_2.draft_all(naive_draft = True,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_2.roto_placement))

Placement  = 12


In [ ]:
# This time do the full-simulation, recommended picks version, i.e.; naive_draft = False
# Remember, it will try every remaining open position and pick the one whose resulting placement is highest.  

zips_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zips_2.draft_all(naive_draft = False,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_2.roto_placement))

Team 2 picking J.T. Realmuto for C
Team 2 picking Josh Hader for RP
Swapping UTIL with 2B
Team 2 picking Jose Altuve for 2B
Team 2 picking Ketel Marte for SS


In [14]:
# Lets compare Naive True/False for the 7th pick

zipsNaive_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zipsNaive_7.draft_all(naive_draft = True,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_7.roto_placement))

Placement  = 1


In [15]:
zips_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zips_7.draft_all(naive_draft = False,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_7.roto_placement))

Team 7 picking Trea Turner for BN
Team 7 picking Freddie Freeman for 1B
Team 7 picking Xander Bogaerts for 2B
Team 7 picking J.T. Realmuto for C
Team 7 picking Anthony Rizzo for UTIL
Team 7 picking Victor Robles for BN
Team 7 picking Brandon Woodruff for P
Team 7 picking Josh Donaldson for 3B
Team 7 picking Carlos Carrasco for SP
Team 7 picking Jesus Luzardo for P
Team 7 picking Max Kepler for OF
Team 7 picking Miguel Sano for BN
Team 7 picking Justin Turner for BN
Team 7 picking Tommy Edman for OF
Team 7 picking Luke Weaver for SP
Team 7 picking Jake Odorizzi for P
Team 7 picking Luke Voit for BN
Team 7 picking Dansby Swanson for SS
Team 7 picking Avisail Garcia for OF
Team 7 picking Mychal Givens for RP
Team 7 picking Seth Lugo for RP
Placement  = 1


In [18]:
# And for the 11th pick

zipsNaive_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zipsNaive_11.draft_all(naive_draft = True,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_11.roto_placement))

Placement  = 6


In [19]:
zips_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zips_11.draft_all(naive_draft = False,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_11.roto_placement))

Team 11 picking Rafael Devers for 3B
Team 11 picking DJ LeMahieu for 1B
Team 11 picking Bryce Harper for OF
Team 11 picking Chris Sale for SP
Team 11 picking Jeff McNeil for 2B
Team 11 picking Michael Conforto for OF
Team 11 picking Hunter Dozier for OF
Team 11 picking Andrew Benintendi for UTIL
Team 11 picking Tommy Edman for BN
Team 11 picking Shogo Akiyama for BN
Team 11 picking J.T. Realmuto for C
Team 11 picking Mike Soroka for SP
Team 11 picking Elvis Andrus for SS
Team 11 picking Tony Gonsolin for RP
Team 11 picking Jean Segura for BN
Team 11 picking Tim Anderson for BN
Team 11 picking Oliver Drake for RP
Team 11 picking Brad Peacock for P
Team 11 picking Ty Buttrey for P
Team 11 picking Cam Bedrosian for P
Team 11 picking Matt Magill for BN
Placement  = 1


In [20]:
# We can do simlated drafts for different projection systems.  Above was all with ZiPs.  Lets try a Steamer, with custom league settings.

number_teams = 6
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2}
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams, roster_spots=roster_spots)
steam_custom.draft_all(naive_draft = True,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_custom.roto_placement))

Placement  = 2


In [21]:
# And compare Naive to the recommendations from the simulation

roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2}
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams, roster_spots=roster_spots)
steam_custom.draft_all(naive_draft = False,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_custom.roto_placement))

Team 2 picking Mookie Betts for OF
Team 2 picking J.T. Realmuto for C
Team 2 picking Charlie Blackmon for OF
Team 2 picking Jack Flaherty for SP
Team 2 picking Rafael Devers for 3B
Team 2 picking Kris Bryant for BN
Team 2 picking Kirby Yates for RP
Team 2 picking Aaron Judge for OF
Team 2 picking Jorge Polanco for SS
Team 2 picking George Springer for UTIL
Team 2 picking Pete Alonso for 1B
Team 2 picking Marcus Semien for BN
Team 2 picking Will Harris for RP
Team 2 picking Eduardo Escobar for 2B
Team 2 picking Emilio Pagan for P
Team 2 picking Luis Castillo for SP
Team 2 picking Yusmeiro Petit for P
Team 2 picking Ryan Pressly for P
Placement  = 1


In [22]:
# And lets compare Naive True/False for different draft positions

steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = True,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_2.roto_placement))

Placement  = 8


In [23]:
steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = False,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_2.roto_placement))

Team 2 picking Josh Hader for RP
Team 2 picking Bryce Harper for OF
Team 2 picking Stephen Strasburg for SP
Team 2 picking Whit Merrifield for 2B
Team 2 picking Anthony Rizzo for 1B
Team 2 picking Aroldis Chapman for RP
Team 2 picking Eddie Rosario for C
Team 2 picking Mike Soroka for SP
Team 2 picking Luis Robert for OF
Team 2 picking Miguel Sano for 3B
Team 2 picking Carlos Santana for UTIL
Team 2 picking Elvis Andrus for SS
Team 2 picking Kyle Schwarber for OF
Team 2 picking Andrew Heaney for P
Team 2 picking Gavin Lux for BN
Team 2 picking Mike Foltynewicz for P
Team 2 picking Paul DeJong for BN
Team 2 picking Justin Upton for BN
Team 2 picking Ryan Yarbrough for P
Team 2 picking Tom Murphy for BN
Team 2 picking David Peralta for BN
Placement  = 3


In [24]:
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = True,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_7.roto_placement))

Placement  = 7


In [25]:
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = False,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_7.roto_placement))

Team 7 picking Josh Hader for RP
Team 7 picking Kirby Yates for RP
Team 7 picking Kris Bryant for 3B
Team 7 picking George Springer for OF
Team 7 picking Manny Machado for SS
Team 7 picking Eloy Jimenez for OF
Team 7 picking Josh Bell for 1B
Team 7 picking Matt Chapman for UTIL
Team 7 picking Shohei Ohtani for SP
Team 7 picking Andrew Benintendi for OF
Team 7 picking Oscar Mercado for BN
Team 7 picking Danny Santana for 2B
Team 7 picking Wilson Ramos for C
Team 7 picking Sean Manaea for SP
Team 7 picking Edwin Encarnacion for BN
Team 7 picking Scott Kingery for BN
Team 7 picking J.D. Davis for BN
Team 7 picking Andrew McCutchen for BN
Team 7 picking Yonny Chirinos for P
Team 7 picking Seth Lugo for P
Team 7 picking Ross Stripling for P
Placement  = 1


In [26]:
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = True,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_11.roto_placement))

Placement  = 12


In [27]:
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = False,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_11.roto_placement))

Team 11 picking Freddie Freeman for 1B
Team 11 picking Justin Verlander for SP
Team 11 picking Xander Bogaerts for SS
Swapping UTIL with OF
Team 11 picking Austin Meadows for OF
Team 11 picking Paul Goldschmidt for UTIL
Team 11 picking Yoan Moncada for 3B
Team 11 picking Mike Moustakas for 2B
Team 11 picking Marcus Semien for BN
Team 11 picking Michael Conforto for BN
Team 11 picking Michael Brantley for SP
Team 11 picking Salvador Perez for C
Team 11 picking Corey Seager for BN
Team 11 picking Mike Minor for P
Team 11 picking Tommy Edman for OF
Team 11 picking Keone Kela for RP
Team 11 picking Bryan Reynolds for OF
Team 11 picking Joe Musgrove for P
Team 11 picking Mark Melancon for RP
Team 11 picking Rougned Odor for BN
Team 11 picking Joc Pederson for BN
Team 11 picking Cole Hamels for P
Placement  = 1


In [28]:
# And finally, 7th pick using TheBat projection system. 

thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = True,shuffle_picks = shuffle_picks)
print('Placement  = '+str(thebat_7.roto_placement))

Placement  = 7


In [29]:
thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = False,shuffle_picks = shuffle_picks)
print('Placement  = '+str(thebat_7.roto_placement))

Team 7 picking Jose Altuve for 2B
Team 7 picking J.T. Realmuto for C
Team 7 picking Starling Marte for OF
Team 7 picking Kris Bryant for 3B
Team 7 picking Eddie Rosario for OF
Team 7 picking Eloy Jimenez for OF
Team 7 picking Rhys Hoskins for 1B
Team 7 picking Carlos Santana for UTIL
Team 7 picking Taylor Rogers for RP
Team 7 picking Gavin Lux for BN
Team 7 picking Andrew McCutchen for BN
Team 7 picking Mike Minor for SP
Team 7 picking Christian Walker for BN
Team 7 picking Bryan Reynolds for BN
Team 7 picking Kenta Maeda for SP
Team 7 picking Carlos Martinez for RP
Team 7 picking Jesus Luzardo for P
Team 7 picking Tommy La Stella for BN
Team 7 picking Dansby Swanson for SS
Team 7 picking Mike Fiers for P
Team 7 picking Miles Mikolas for P
Placement  = 1


In [30]:
# If you want to see the picks happening in real-time, choose silent=False

zips = Draft(player_rankings['ZiPS'])
zips.draft_all(silent=False,shuffle_picks = shuffle_picks)

TypeError: draft_all() got an unexpected keyword argument 'silent'